In [26]:
# Primero se importan las librerias a usar
import numpy as np
import sympy as sp
import heapq as he
import math as mt
from math import pi,cos,sin
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
# EJERCICIO 1

#Datos
Codigo = "2191437"

#Factor de carga de la distribución de correas y los materiales 
X = sum([int(i) for i in Codigo])

#Factor de amplitud debido a carga viva y de viento
Y = X/3

#Carga máxima W distribuida en Kn/m
W = (0.005*X) + (0.01*Y)

#la tensión a la que se someterá el cable que pende del extremo B
P = 0.05*X
Angulo = pi/4

#Las longitudes de aplicación de cada carga distribuida m 
L1 = 0.05*X
L2 = L1*1.5
L3 = 0.5*L1
L4 = L1 + L2 + L3 

#Esfuerzo admisible Mpa
Esf_adm = 250

In [11]:
# Se van a puntualizar las cargas distribuidas

#Carga triángular puntualizada kN 
#Posición m
Wt1 = (L1*W)/2
X1 = L1 - L1/3

#Carga cuadrada puntualizada kN 
#Posición m
Wc = L2*W
X2 = L1 + L2/2 

#Carga triángular puntualizada kN (numero dos)
#posición m
Wt2 = (L3*W)/2
X3 = L1 + L2 + L3/3 

# Definir simbolos
Ay, Ma, X = sp.symbols("Ay Ma X")

# Calcular reacciones
Suma_Ma = Ma - X1*Wt1 - X2*Wc - X3*Wt2 + L4*P*sin(Angulo)
Ec_Ma = sp.Eq (Suma_Ma,0)
Momento = sp.solve(Ec_Ma)

Fy = Ay -Wt1 - Wc - Wt2 + P*sin(Angulo)
Ec_Fy = sp.Eq(Fy,0)
Reaccion = sp.solve(Ec_Fy)

In [12]:
# Se imprimen los resultados de Ma y Ay

print(f"Ma = {Momento[0]} kN-m")
print(f"Ay = {Reaccion[0]} kN")

Ma = -2.37962976363745 kN-m
Ay = -0.271156654601839 kN


In [14]:
# Funcion de carga para el tramo 1
W1 = ((W-0)/(L1-0))*X 
W1

0.166666666666667*X

In [15]:
# Funcion de carga para el tramo 3
W3 = ((0-W)/(L3-0))*X + W
W3

0.225 - 0.333333333333333*X

In [17]:
# Calculo de fuerza cortante para el tramo 1 (0 <= X <= L1)
V1 = -sp.integrate(W1,X) + Reaccion[0]
V1_L1 = V1.subs(X,L1)
V1

-0.0833333333333333*X**2 - 0.271156654601839

In [18]:
# Calculo de momento para el tramo 1 (0 <= X <= L1)
M1 = sp.integrate(V1,X)- Momento[0]
M1_0 = M1.subs(X,0)
M1_L1 = M1.subs(X,L1)
M1

-0.0277777777777778*X**3 - 0.271156654601839*X + 2.37962976363745

In [20]:
# Calculo de fuerza cortante para el tramo 2 (0 <= X <= L2)
V2 = -sp.integrate(W,X) + V1_L1
V2_L2 = V2.subs(X,L2)
V2

-0.225*X - 0.423031654601839

In [21]:
# Calculo de momento para el tramo 2 (0 <= X <= L2)
M2 = sp.integrate(V2,X) + M1_L1
M2_L2 = M2.subs(X,L2)
M2

-0.1125*X**2 - 0.423031654601839*X + 1.94522452992497

In [22]:
# Calculo de fuerza cortante para el tramo 3 (0 <= X <= L3)
V3 = -sp.integrate(W3,X) + V2_L2
V3_L3 = V3.subs(X,L3)
V3

0.166666666666667*X**2 - 0.225*X - 0.878656654601839

In [23]:
# Calculo de momento para el tramo 3 (0 <= X <= L3)
M3 = sp.integrate(V3,X) + M2_L2
M3_L3 = M3.subs(X,L3)
M3

0.0555555555555556*X**3 - 0.1125*X**2 - 0.878656654601839*X + 0.627265116856243

In [27]:
#Grafica para el diagrama de V
plt.style.use('ggplot')

#Longitud de la viga
L = np.linspace(0,L4,2000)

#Convertir las expresiones de los tramos para que se puedan evaluar matematicamente 
Tr1 = sp.lambdify(X, V1)(L[L < L1])
Tr2 = sp.lambdify(X, V2)(L[(L >= L1) & (L < L1+L2)] - L1) 
Tr3 = sp.lambdify(X, V3)(L[(L >= L1+L2) & (L <= L4)] - L1 - L2)

#Crear la grafica para el diagrama
plt.plot(L, np.concatenate((Tr1, Tr2, Tr3)), color = "r")
plt.fill_between(L, np.concatenate((Tr1, Tr2, Tr3)), color = "m", alpha=0.25)
plt.xticks(np.linspace(0,3.15,10))

#Generar los ejes coordenados X y Y
plt.axvline(0, color="black")
plt.axhline(0, color="black")

#Escribir el nombre de los ejes coordenados X y Y
plt.xlabel("Longitud [m]")
plt.ylabel("Fuerza Cortante [kN]")

#Colocar el titulo de la grafica
plt.title("Diagrama De Fuerza Cortante")

#Escribir la fuerza cortante maxima y donde ocurre
print(f"La fuerza cortante máxima es de {round(V3_L3,4)} kN, a una distancia de {L4} m.")

La fuerza cortante máxima es de -0.9546 kN, a una distancia de 4.050000000000001 m.


In [28]:
# Diagrama para el momento flector
plt.style.use('ggplot')

#Longitud de la viga
L = np.linspace(0,L4,2000)

#Convertir las expresiones de los tramos para que se puedan evaluar matematicamente
Tr1 = sp.lambdify(X, M1)(L[L < L1])
Tr2 = sp.lambdify(X, M2)(L[(L >= L1) & (L < L1+L2)] - L1) 
Tr3 = sp.lambdify(X, M3)(L[(L >= L1+L2) & (L <= L4)] - L1 - L2)

#Crear la grafica para el diagrama
plt.plot(L, np.concatenate((Tr1, Tr2, Tr3)), color = "r")
plt.fill_between(L, np.concatenate((Tr1, Tr2, Tr3)), color = "m", alpha=0.25)
plt.xticks(np.linspace(0,3.15,10))

#Generar los ejes coordenados X y Y
plt.axvline(0, color="black")
plt.axhline(0, color="black")

#Escribir el nombre de los ejes coordenados X y Y
plt.xlabel("Longitud [m]")
plt.ylabel("Momento [kN-m]")

#Colocar el titulo de la grafica
plt.title("Diagrama De Momento Flector")

#Escribir el momento flector maximo y donde ocurre
print(f"El momento flector máximo es de {round(M1_0,4)} kN-m y ocurre en el empotramiento.")

El momento flector máximo es de 2.3796 kN-m y ocurre en el empotramiento.
